In [1]:
# Import the necessary libraries

import re
from numpy import array
from numpy import asarray
from numpy import zeros

import tensorflow as tf
import keras
import keras.backend as K
from keras.models import Model
from keras.regularizers import L2
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Dropout, Input, LSTM
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

2022-05-01 15:12:01.355493: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-01 15:12:01.355544: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[nltk_data] Downloading package punkt to /home/raghav/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [50]:
# Helper function that cleans the input data and enumerates the labels

def extract(s):
    s = re.sub('\\(', '', s)
    s = re.sub('\\)', '', s)
    s = re.sub('\\s{2,}', ' ', s)
    return s.strip()

labels = {'entails\n': 0, 'contradicts': 1, 'neutral\n': 2}

In [51]:
# Function that reads data and parses data from file

def readFileData(filePath):
    with open(filePath, 'r') as f:
        inputRows = [row.split('\t') for row in f.readlines()[1:]]

    inputPremises = [row[0] for row in inputRows if row[2] in labels]
    inputHypotheses = [row[1] for row in inputRows if row[2] in labels]
    inputLabels = [labels[row[2]] for row in inputRows if row[2] in labels]
    f.close()

    return [inputPremises, inputHypotheses, inputLabels]

In [52]:
# Reading train and test data

trainData = readFileData('./scitail_1.0_train.tsv')
testData = readFileData('./scitail_1.0_test.tsv')
validationData = readFileData('./scitail_1.0_dev.tsv')

In [5]:
# Hyperparameters

maxLen = 32
epochs = 1000
batchSize = 512
gloveDimension = 300
hiddenDimension = 100
regularization = 4e-6

In [54]:
# Tokenizer to generate the vocabulary of the system

tokenizer = Tokenizer()
tokenizer.fit_on_texts(trainData[0] + trainData[1])
vocabSize = len(tokenizer.word_index)+1

In [55]:
# Convert the train data to sequences as per the vocabulary
trainData[0] = tokenizer.texts_to_sequences(trainData[0])
trainData[1] = tokenizer.texts_to_sequences(trainData[1])

# Pad or trim all generated sequences to the same max sentence length
trainData[0] = pad_sequences(trainData[0], maxLen, padding='post')
trainData[1] = pad_sequences(trainData[1], maxLen, padding='post')

# Transform the labels to one-hot encoding
trainData[2] = tf.keras.utils.to_categorical(trainData[2], num_classes=3)

In [56]:
# Convert the test data to sequences as per the vocabulary
testData[0] = tokenizer.texts_to_sequences(testData[0])
testData[1] = tokenizer.texts_to_sequences(testData[1])

# Pad or trim all generated sequences to the same max sentence length
testData[0] = pad_sequences(testData[0], maxLen, padding='post')
testData[1] = pad_sequences(testData[1], maxLen, padding='post')

# Transform the labels to one-hot encoding
testData[2] = tf.keras.utils.to_categorical(testData[2], num_classes=3)

In [57]:
# Convert the validation data to sequences as per the vocabulary
validationData[0] = tokenizer.texts_to_sequences(validationData[0])
validationData[1] = tokenizer.texts_to_sequences(validationData[1])

# Pad or trim all generated sequences to the same max sentence length
validationData[0] = pad_sequences(validationData[0], maxLen, padding='post')
validationData[1] = pad_sequences(validationData[1], maxLen, padding='post')

# Transform the labels to one-hot encoding
validationData[2] = tf.keras.utils.to_categorical(validationData[2], num_classes=3)

In [10]:
embeddingsDict = dict()
glove = open(r'glove.840B.300d.txt', encoding='utf8')

for line in glove:
    records = line.split()
    word = ''.join(records[:-300])
    vectorDimensions = asarray(records[-300:], dtype='float32')
    embeddingsDict[word] = vectorDimensions

glove.close()

In [58]:
# Iterate through the embeddings and store only those that are present in our vocabulary
embeddingsMat = zeros((vocabSize, gloveDimension))
for word, index in tokenizer.word_index.items():
    vec = embeddingsDict.get(word)
    if vec is not None:
        embeddingsMat[index] = vec

In [59]:
# Define the embedding layer for our baseline RNN model
embed = Embedding(vocabSize, gloveDimension, weights=[embeddingsMat], input_length=maxLen, trainable=False)

# As Premise and Hypothesis are distinct and are to be inputted separately, define two inputs and embed
premise = Input(shape=(maxLen,), dtype='int32')
hypothesis = Input(shape=(maxLen,), dtype='int32')

premInput = embed(premise)
hypoInput = embed(hypothesis)

convert = Dense(hiddenDimension, activation='tanh', input_shape=(gloveDimension,))

premInput = convert(premInput)
hypoInput = convert(hypoInput)

In [78]:
# Once the sentence embeddings have been generated, generate a matrix of dimensions maxLen X gloveDimension
# On adding maxLen, we get a single embedding vector of length gloveDimension

rnn = keras.layers.core.Lambda(lambda x: K.sum(x, axis=1), output_shape=hiddenDimension)
#rnn = LSTM(hiddenDimension, dropout=0.2, recurrent_dropout=0.2)

In [72]:
# Apply batch normalization to the two input embeddings separately

premInput = rnn(premInput)
hypoInput = rnn(hypoInput)
premInput = tf.keras.layers.BatchNormalization()(premInput)
hypoInput = tf.keras.layers.BatchNormalization()(hypoInput)

ValueError: Input 0 of layer "lstm_2" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 100)

In [79]:
# Joint is a concatenated embeddings layer, generated from the premise and hypothesis inputs
# Dilution of probability 0.2, to assist in regularization
joint = keras.layers.concatenate([premInput, hypoInput])
joint = Dropout(0.2)(joint)
for i in range(3):
    joint = Dense(2*hiddenDimension, activation='tanh', kernel_regularizer=L2(regularization))(joint)
    joint = Dropout(0.2)(joint)
    joint = tf.keras.layers.BatchNormalization()(joint)

# 3 layers of the TanH activation function, along with L2 regularization.
# The final decision is based on the Softmax function
pred = Dense(3, activation='softmax')(joint)

In [80]:
# Defining the final models input and output format, as well as compilation parameters

model = Model(inputs=[premise, hypothesis], outputs=pred)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [81]:
# Fitting the model using the train data

callback = EarlyStopping(monitor='val_loss', min_delta=0, patience=20)
model.fit([array(trainData[0]), array(trainData[1])], array(trainData[2]), batch_size=batchSize, epochs=epochs, callbacks=[callback], validation_data=[[array(validationData[0]), array(validationData[1])], array(validationData[2])])

Epoch 1/1000
47/47 [==============================] - 52s 878ms/step - loss: 0.6214 - accuracy: 0.8007 - val_loss: 0.8246 - val_accuracy: 0.7360
Epoch 2/1000
47/47 [==============================] - 37s 786ms/step - loss: 0.4134 - accuracy: 0.8620 - val_loss: 0.6789 - val_accuracy: 0.7160
Epoch 3/1000
47/47 [==============================] - 37s 781ms/step - loss: 0.3447 - accuracy: 0.8707 - val_loss: 0.8857 - val_accuracy: 0.7145
Epoch 4/1000
47/47 [==============================] - 38s 815ms/step - loss: 0.3135 - accuracy: 0.8741 - val_loss: 0.8190 - val_accuracy: 0.7122
Epoch 5/1000
47/47 [==============================] - 37s 791ms/step - loss: 0.2997 - accuracy: 0.8788 - val_loss: 1.0625 - val_accuracy: 0.7068
Epoch 6/1000
 3/47 [>.............................] - ETA: 35s - loss: 0.3205 - accuracy: 0.8659

KeyboardInterrupt: 

In [82]:
# Evaluating accuracy on the trained model

loss, acc = model.evaluate([array(testData[0]), array(testData[1])], array(testData[2]), batch_size=256)
print('Loss = ', loss)
print('Acc = ', acc)

9/9 [==============================] - 1s 115ms/step - loss: 0.7257 - accuracy: 0.7162
Loss =  0.7256543040275574
Acc =  0.7162352800369263


In [83]:
model.save('./lmaosci')

INFO:tensorflow:Assets written to: ./lmaosci/assets


INFO:tensorflow:Assets written to: ./lmaosci/assets


In [ ]:
ccc = keras.models.load_model('./lstm')
l, a = ccc.evaluate([array(testData[0]), array(testData[1])], array(testData[2]), batch_size=256)
print(a)

39/39 [==============================] - 4s 78ms/step - loss: 0.5769 - accuracy: 0.7730
0.7730048894882202
